0. 에이전트를 사용
1. wikipediaRetriver 사용
2. DuckDuckGoSearchAPIWrapper 사용
3. 근데 웹사이트의 텍스트를 스크랩 및 추출?
4. 결과를 txt로 저장

에이전트 생성
에이전트 툴 생성
에이전트 툴에는 wikipediaRetriver를 사용하는 함수랑 DuckDuckGoSearchAPIWrapper를 사용하는 함수. 두 함수를 묶는 함수를 툴로 사용
두 함수를 묶는 함수는 앞선 두 함수의 결과를 파일로 저장해야함.

In [26]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.retrievers import WikipediaRetriever
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.document_loaders import WebBaseLoader

llm = ChatOpenAI(
    temperature=0.1,
    # model="gpt-4o"
    )

In [34]:
# 데이터 저장용 툴

def save_the_file(docs):
    f = open("./file.txt","w",encoding="utf-8")
    f.write(docs)

# WikipediaRetriever 사용한 tool

def Wikipedia(keyword):
    rs = "Wikipedia Result\n\n"
    retriver = WikipediaRetriever(top_k_results=3, lang="ko")
    data_list = retriver.get_relevant_documents(keyword)
    for page_content in data_list:
        rs += f"{page_content.page_content} \n\n"
    return rs

class WikipediaToolArgsSchema(BaseModel):
    keyword: str = Field(description="'Search Keyword' to Use for Wikipedia Search")

class WikipediaTool(BaseTool):
    name = "Wikipedia"
    description = """
    A tool used by Wikipedia to search for specific keywords. Use 'keyword' as a parameter.
    """
    args_schema: Type[WikipediaToolArgsSchema] = WikipediaToolArgsSchema

    def _run(self, keyword):
        rs = Wikipedia(keyword)
        save_the_file(rs)
        return rs
    

# DuckDuckGoSearchAPIWrapper 사용한 tool

# 해당 함수를 통해 duckduckgo가 찾아낸 사이트의 정보를 추출하려했으나, 매 시도마다 너무 많은양의 문서 데이터로 에러가 발생.
# 이로인해 간단한 데이터와 출처만 저장하는 식으로 지정
def parse_DuckDuckGo(docs):
    data_list = []
    return_data = []
    datas = docs.split("https://")
    data = datas[1].split(")\n")[0].split(",")[0]
    data_list.append(f"https://{data}")
    for url in data_list:
        loader = WebBaseLoader(web_path=url)
        documents = loader.load()
        for page_content in documents:
            return_data.append(page_content.page_content)
    return return_data

class DuckDuckGoSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: WW2"
    )

class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = """
    A tool that uses DuckDuckGo to search for specific keywords and return the results to the user.
    """
    args_schema: Type[
        DuckDuckGoSearchToolArgsSchema
    ] = DuckDuckGoSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        rs = ddg.run(query)
        save_the_file(rs)
        return rs


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaTool(),
        DuckDuckGoSearchTool()
    ],
)

prompt = "Tell me about WW2 using Wikipedia"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `Wikipedia` with `{'keyword': 'World War II'}`


Wikipedia Result

제2차 세계 대전(한국 한자: 第二次世界大戰, 영어: World War II, WWII, WW2)은 1939년 9월 1일부터 1945년 9월 2일까지 일어났던 세계 대전이다. 당시 강대국 전부와 세계 대부분의 국가가 전쟁에 개입해 연합국과 추축국이라는 적대적인 두 군사동맹이 생겨났다. 제2차 세계 대전은 30개국 이상에서 1억명이 넘는 군인이 직접 참전한 총력전이다. 세계 대전 참전국은 총체적인 경제적, 산업적, 과학적 역량을 전부 전쟁 수행에 쏟아 부어 민간 자원과 군사 자원의 경계가 모호해졌다. 항공기도 전쟁에서 중요한 역할을 맡아 항공기를 통해 인구 밀집지대의 전략폭격이 수행되었고 유일하게 실전 사용된 핵무기 2발도 배치하고 투하하는 데 항공기가 사용되었다. 제2차 세계 대전은 인류 역사상 사망자가 가장 많은 전쟁으로 총 사망자는 7,500만명 정도로 추정되며 이 중 대부분이 민간인이었다. 또한 수 천만 명이 홀로코스트를 포함한 대량학살, 기아, 학살, 질병 등으로 사망했다. 제2차 세계 대전에서 추축국이 패배해 독일과 일본이 연합국에게 점령당했으며 전쟁범죄를 일으킨 독일과 일본의 주요 지도자에 대한 재판이 열렸다.
제2차 세계 대전의 원인으로는 여러 가지가 꼽히고 논쟁도 많지만, 대표적으로는 제2차 이탈리아-에티오피아 전쟁, 스페인 내전, 중일전쟁, 소련-일본 국경 분쟁과 제1차 세계 대전 이후 유럽의 파시즘 대두와 긴장 고조가 꼽힌다. 제2차 세계 대전의 개전일은 통상 아돌프 히틀러의 나치 독일이 폴란드 침공을 개시한 1939년 9월 1일로 본다. 뒤이어 영국과 프랑스가 9월 3일 독일에게 선전포고했다. 1939년 8월 맺어졌던 독일-소련 불가침 조약에 따라 독일과 소련은 폴란드를 분할하고 핀란드, 리투아니아, 라트비아, 에스토니아, 루마니아의

{'input': 'Tell me about WW2 using Wikipedia',
 'output': 'The Second World War (WW2) was a global conflict that took place from September 1, 1939, to September 2, 1945. It involved major powers and most countries around the world, leading to the formation of two hostile military alliances known as the Allies and the Axis powers. Over 100 million soldiers from more than 30 countries participated in the war, making it a total war involving significant economic, industrial, and scientific resources.\n\nThe war saw the use of strategic bombing in densely populated areas, including the deployment of nuclear weapons for the first time. It resulted in an estimated total death toll of around 75 million people, with a majority being civilians. Millions died due to mass killings, famine, diseases, and other atrocities, including the Holocaust.\n\nThe Axis powers, including Germany and Japan, were defeated by the Allies, leading to their occupation by the victorious nations. War crimes trials we